In [1]:
%use ktor-client
%use coroutines
%use serialization

import io.ktor.client.*
import io.ktor.client.call.*
import io.ktor.client.engine.cio.*
import io.ktor.client.plugins.contentnegotiation.*
import io.ktor.client.request.*
import io.ktor.client.statement.bodyAsChannel
import io.ktor.serialization.kotlinx.json.*
import io.ktor.utils.io.ByteReadChannel
import io.ktor.utils.io.jvm.javaio.copyTo
import kotlinx.serialization.json.*
import kotlinx.coroutines.*
import java.io.File
import java.net.URI
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.coroutineScope
import kotlinx.coroutines.delay
import kotlinx.coroutines.launch
import kotlinx.coroutines.runBlocking
import java.io.FileInputStream
import java.io.FileOutputStream
import java.util.zip.ZipEntry
import java.util.zip.ZipInputStream
import java.nio.charset.Charset

In [2]:
// Step 1: Create folders
val currentDir = System.getProperty("user.dir")

val doomRPGFolder = File(currentDir, "DoomRPG")
doomRPGFolder.mkdirs()

val gameFolder = File(doomRPGFolder, "Game")
gameFolder.mkdirs()
val tempFolder = File(doomRPGFolder, "Temporary")
tempFolder.mkdirs()

val filesFolder = File(tempFolder, "Files")
filesFolder.mkdirs()
val brewBinaryFolder = File(tempFolder, "Brew Binary")
brewBinaryFolder.mkdirs()
val desktopBinaryFolder = File(tempFolder, "Desktop Binary")
desktopBinaryFolder.mkdirs()

// Step 2: Download necessary files
val brewBinaryUrl = "https://archive.org/download/doomrpg_brew/doomrpg.zip"
val brewBinary = File(filesFolder, "doomrpg.zip")

runBlocking {
    val client = HttpClient(CIO)

    client.use {
        println("Downloading Brew binary via Ktor...")

        try {
            // Делаем запрос
            val response = it.get(brewBinaryUrl) {
                // Хорошим тоном считается добавить User-Agent
                header("User-Agent", "Kotlin-Notebook-Downloader")
            }

            if (response.status.value in 200..299) {
                // Получаем поток данных
                val channel: ByteReadChannel = response.bodyAsChannel()

                // Открываем файл для записи и копируем в него поток
                brewBinary.outputStream().use { fileStream ->
                    channel.copyTo(fileStream)
                }

                println("✅ Brew binary downloaded → ${brewBinary.absolutePath}")
            } else {
                println("❌ Ошибка сервера: ${response.status}")
            }
        } catch (e: Exception) {
            println("❌ Ошибка при загрузке: ${e.localizedMessage}")
        }
    }
}

val repo = "Erick194/DoomRPG-RE"
lateinit var desktopBinary: File

runBlocking {
    // Создаем клиент и сразу настраиваем JSON
    val client = HttpClient(CIO) {
        install(ContentNegotiation) {
            // Игнорируем лишние поля, чтобы не было ошибок
            json(Json { ignoreUnknownKeys = true })
        }
    }

    lateinit var desktopBinaryUrl: String

    // Используем .use, чтобы клиент корректно закрылся после работы
    try {
        val response = client.get("https://api.github.com/repos/$repo/releases/latest") {
            // GitHub API обязательно требует User-Agent
            header("User-Agent", "Kotlin-Notebook")
        }.body<JsonObject>()

        // Пробираемся через JSON: assets -> [0] -> browser_download_url
        desktopBinaryUrl = response["assets"]
            ?.jsonArray
            ?.firstOrNull()
            ?.jsonObject
            ?.get("browser_download_url")
            ?.jsonPrimitive
            ?.content ?: ""

        if (desktopBinaryUrl.isNotEmpty()) {
            println("✅ Прямая ссылка получена:\n$desktopBinaryUrl")
        } else {
            println("❌ Ссылка не найдена (возможно, в релизе нет прикрепленных файлов).")
        }
    } catch (exception: Exception) {
        println("❌ Ошибка запроса: ${exception.message}")
    }

    try {
        println("Downloading Desktop Binary via Ktor...")

        // Делаем запрос
        if (desktopBinaryUrl.isNotEmpty()) {
            val response = client.get(desktopBinaryUrl) {
                // Хорошим тоном считается добавить User-Agent
                header("User-Agent", "Kotlin-Notebook-Downloader")
            }

            if (response.status.value in 200..299) {
                // Получаем поток данных
                val channel: ByteReadChannel = response.bodyAsChannel()

                // Открываем файл для записи и копируем в него поток

                // val brewBinary = File(filesFolder, "doomrpg.zip")
                desktopBinary = File(filesFolder, "DoomRPG_Port_latest.zip")

                desktopBinary.outputStream().use { fileStream ->
                    channel.copyTo(fileStream)
                }

                println("✅ Desktop binary downloaded → ${desktopBinary.absolutePath}")
            } else {
                println("❌ Ошибка сервера: ${response.status}")
            }
        }
    } catch (e: Exception) {
        println("❌ Ошибка при загрузке: ${e.localizedMessage}")
    }
}

// Step 3: Extract archives

// Unzip function (if not already defined earlier)
fun unzip(zipFile: File, destDir: File) {
    destDir.mkdirs()
    ZipInputStream(FileInputStream(zipFile)).use { zis ->
        var entry: ZipEntry? = zis.nextEntry
        while (entry != null) {
            val newFile = File(destDir, entry.name)
            if (entry.isDirectory) {
                newFile.mkdirs()
            } else {
                newFile.parentFile.mkdirs()
                FileOutputStream(newFile).use { fos ->
                    zis.copyTo(fos)
                }
            }
            entry = zis.nextEntry
        }
    }
}

println("Extracting Brew binary...")
unzip(brewBinary, brewBinaryFolder)
println("Extracted to → $brewBinaryFolder")

println("Extracting Desktop binary...")
unzip(desktopBinary, desktopBinaryFolder)

// Special handling: Flatten "DoomRPG" subfolder contents to root (Temporary/Desktop Binary)
val doomRpgSubFolder = File(desktopBinaryFolder, "DoomRPG")
if (doomRpgSubFolder.exists() && doomRpgSubFolder.isDirectory) {
    doomRpgSubFolder.listFiles()?.forEach { child ->
        child.copyRecursively(File(desktopBinaryFolder, child.name), overwrite = true)
    }
    doomRpgSubFolder.deleteRecursively()
    println("Flattened DoomRPG subfolder contents to → $desktopBinaryFolder")
} else {
    println("Note: No 'DoomRPG' subfolder found after extraction.")
}

println("Desktop extraction complete → $desktopBinaryFolder")

// ────────────────────────────────────────────────
// Ready for next steps:
// Contents of brew_binary are now in Temporary/Brew Binary (e.g., doomrpg.bar)
// Contents of desktop_binary (from DoomRPG subfolder) are now directly in Temporary/Desktop Binary (e.g., DoomRPG.exe, BarToZip.exe)
// ────────────────────────────────────────────────

// Step 4: Final preparations

// Copy content of Temporary/Desktop Binary into DoomRPG/Game
println("Copying Desktop Binary contents to Game folder...")
desktopBinaryFolder.copyRecursively(gameFolder, overwrite = true)
println("Copied to → $gameFolder")

// Copy doomrpg.bar from Temporary/Brew Binary into DoomRPG/Game
val barSource = File(brewBinaryFolder, "doomrpg.bar")
val barDest = File(gameFolder, "doomrpg.bar")
if (barSource.exists()) {
    barSource.copyTo(barDest, overwrite = true)
    println("Copied doomrpg.bar → $barDest")
} else {
    println("Warning: doomrpg.bar not found in $brewBinaryFolder")
}

// Run BarToZip.exe in DoomRPG/Game (silently, with timeout-based key detection)
val barToZipExe = File(gameFolder, "BarToZip.exe")

if (barToZipExe.exists()) {
    println("Running BarToZip.exe silently in $gameFolder...")
    val pbBarToZip = ProcessBuilder(barToZipExe.absolutePath)
    pbBarToZip.directory(gameFolder)
    val process = pbBarToZip.start()

    runBlocking {
        val stdin = process.outputStream.bufferedWriter(Charset.forName("UTF-8"))
        val stdout = process.inputStream.bufferedReader(Charset.forName("UTF-8"))
        val stderr = process.errorStream.bufferedReader(Charset.forName("UTF-8"))

        var lastLineTime = System.currentTimeMillis()
        var keySent = false

        // Optimized coroutine for stdout: Track last output time
        val outputJob = launch(Dispatchers.IO) {
            try {
                coroutineScope {
                    while (true) {
                        val line = stdout.readLine() ?: break
                        lastLineTime = System.currentTimeMillis()  // Update timestamp on any output
                        // Silent: Do not print
                    }
                }
            } catch (e: Exception) {
                // Ignore
            } finally {
                try { stdout.close() } catch (e: Exception) {}
            }
        }

        // Optimized coroutine for stderr: Read silently
        val errorJob = launch(Dispatchers.IO) {
            try {
                coroutineScope {
                    while (true) {
                        stderr.readLine() ?: break
                        // Ignore completely
                    }
                }
            } catch (e: Exception) {
                // Ignore
            } finally {
                try { stderr.close() } catch (e: Exception) {}
            }
        }

        // Key-sending monitor: Check periodically for pause in output
        val keyMonitorJob = launch(Dispatchers.Default) {
            while (!keySent) {
                delay(1000L)  // Check every 1s
                val now = System.currentTimeMillis()
                if (now - lastLineTime > 3000L) {  // 3s no output → assume waiting for key
                    println("Detected pause in output; sending key programmatically...")
                    stdin.write("\r\n")
                    stdin.flush()
                    keySent = true
                    delay(2000L)  // Wait 2s for process to react
                    break
                }
            }
        }

        // Wait for process with timeout
        val exitCodeDeferred = async {
            try {
                process.waitFor()
            } catch (e: InterruptedException) {
                process.destroy()
                1
            }
        }

        val timeoutJob = async {
            delay(60000L)  // 60s total timeout
            if (process.isAlive) {
                process.destroyForcibly()
                println("Warning: Total timeout; force-killed BarToZip.exe.")
                1
            } else {
                0
            }
        }

        val exitCode = try {
            exitCodeDeferred.await()
        } catch (e: Exception) {
            timeoutJob.await()
        }

        // Cleanup stdin
        try {
            stdin.close()
        } catch (e: Exception) {}

        // Cancel all jobs
        listOf(outputJob, errorJob, keyMonitorJob).forEach { it.cancel() }

        // Join with timeout
        listOf(outputJob, errorJob).forEach { job ->
            try {
                withTimeout(5000L) {
                    job.join()
                }
            } catch (e: TimeoutCancellationException) {
                // Ignore timeout on join
            }
        }

        // Enhanced termination: Use taskkill by PID if still alive (Windows-specific)
        if (process.isAlive) {
            val pid = try {
                (process as java.lang.ProcessHandle).pid()  // Java 9+ API
            } catch (e: Exception) {
                -1L  // Fallback if PID not available
            }
            if (pid > 0) {
                val taskkillPb = ProcessBuilder("taskkill", "/f", "/pid", pid.toString())
                taskkillPb.directory(gameFolder)
                val taskkillProcess = taskkillPb.start()
                val taskkillExit = taskkillProcess.waitFor()
                println("Taskkill by PID ($pid) completed with exit code: $taskkillExit")
                if (taskkillExit != 0) {
                    println("Warning: taskkill by PID failed.")
                }
            } else {
                // Fallback: Kill by image name
                val fallbackPb = ProcessBuilder("taskkill", "/f", "/im", "BarToZip.exe")
                fallbackPb.directory(gameFolder)
                val fallbackProcess = fallbackPb.start()
                fallbackProcess.waitFor()
                println("Fallback taskkill by image name completed.")
            }

            // Final check after kill
            delay(1000L)
            if (process.isAlive) {
                println("Critical: Process still alive after taskkill. Manual intervention may be needed.")
            }
        }

        if (exitCode != 0 && !keySent) {
            println("Note: Process exited without sending key (possible early termination).")
        }

        println("BarToZip.exe completed silently.")
    }
} else {
    println("Error: BarToZip.exe not found in $gameFolder")
    println("Ensure the Desktop Binary extraction includes this executable.")
}


Detected pause in output; sending key programmatically...
BarToZip.exe completed silently.
